## 08-6 RNN을 이용한 텍스트 생성

### by RNN

In [1]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [9]:
# 예제 문장 저장
text = """경마장에 있는 말이 뛰고 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다"""

In [10]:
text

'경마장에 있는 말이 뛰고 있다\n그의 말이 법이다\n가는 말이 고와야 오는 말이 곱다'

In [11]:
# 단어집합 생성, 크기 확인
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
vocab_size = len(tokenizer.word_index) + 1    # 패딩을 위한 0을 고려해서 단어집합의 크기에 +1을 해줌
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 12


In [12]:
print(tokenizer.word_index)  # 단어와 정수 인덱스 출력

{'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고와야': 9, '오는': 10, '곱다': 11}


In [13]:
text

'경마장에 있는 말이 뛰고 있다\n그의 말이 법이다\n가는 말이 고와야 오는 말이 곱다'

In [14]:
# 훈련 데이터 생성
sequences = list()
for line in text.split('\n'): # 줄바꿈 문자를 기준으로 문장 토큰화
    encoded = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

print('학습에 사용할 샘플의 개수: %d' % len(sequences))

학습에 사용할 샘플의 개수: 11


In [16]:
sequences

[[2, 3],
 [2, 3, 1],
 [2, 3, 1, 4],
 [2, 3, 1, 4, 5],
 [6, 1],
 [6, 1, 7],
 [8, 1],
 [8, 1, 9],
 [8, 1, 9, 10],
 [8, 1, 9, 10, 1],
 [8, 1, 9, 10, 1, 11]]

각 데이터에서 맨 우측에 있는 단어가 레이블로 분리되어야 함

In [17]:
# 가장 긴 샘플 길이 기준으로 길이 맞추기
max_len = max(len(l) for l in sequences) # 모든 샘플에서 길이가 가장 긴 샘플의 길이 출력
print('샘플의 최대 길이 : {}'.format(max_len))

샘플의 최대 길이 : 6


In [18]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')

In [19]:
sequences

array([[ 0,  0,  0,  0,  2,  3],
       [ 0,  0,  0,  2,  3,  1],
       [ 0,  0,  2,  3,  1,  4],
       [ 0,  2,  3,  1,  4,  5],
       [ 0,  0,  0,  0,  6,  1],
       [ 0,  0,  0,  6,  1,  7],
       [ 0,  0,  0,  0,  8,  1],
       [ 0,  0,  0,  8,  1,  9],
       [ 0,  0,  8,  1,  9, 10],
       [ 0,  8,  1,  9, 10,  1],
       [ 8,  1,  9, 10,  1, 11]])

In [20]:
# 마지막 단어는 레이블로 분리
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]

In [21]:
print(X)
print(y)

[[ 0  0  0  0  2]
 [ 0  0  0  2  3]
 [ 0  0  2  3  1]
 [ 0  2  3  1  4]
 [ 0  0  0  0  6]
 [ 0  0  0  6  1]
 [ 0  0  0  0  8]
 [ 0  0  0  8  1]
 [ 0  0  8  1  9]
 [ 0  8  1  9 10]
 [ 8  1  9 10  1]]
[ 3  1  4  5  1  7  1  9 10  1 11]


In [22]:
# 레이블 one hot encoding
y = to_categorical(y, num_classes=vocab_size)
print(y)

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


#### RNN 모델 설계

In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

In [30]:
embedding_dim = 10
hidden_units = 32

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(SimpleRNN(hidden_units))
model.add(Dense(vocab_size, activation='softmax'))    #모든 가능한 단어 중 하나를 예측 -> 다중 클래스 분류
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)

Epoch 1/200
1/1 - 1s - loss: 2.4874 - accuracy: 0.0909 - 655ms/epoch - 655ms/step
Epoch 2/200
1/1 - 0s - loss: 2.4773 - accuracy: 0.0909 - 2ms/epoch - 2ms/step
Epoch 3/200
1/1 - 0s - loss: 2.4673 - accuracy: 0.1818 - 4ms/epoch - 4ms/step
Epoch 4/200
1/1 - 0s - loss: 2.4573 - accuracy: 0.1818 - 4ms/epoch - 4ms/step
Epoch 5/200
1/1 - 0s - loss: 2.4473 - accuracy: 0.1818 - 3ms/epoch - 3ms/step
Epoch 6/200
1/1 - 0s - loss: 2.4370 - accuracy: 0.3636 - 2ms/epoch - 2ms/step
Epoch 7/200
1/1 - 0s - loss: 2.4264 - accuracy: 0.3636 - 3ms/epoch - 3ms/step
Epoch 8/200
1/1 - 0s - loss: 2.4155 - accuracy: 0.3636 - 4ms/epoch - 4ms/step
Epoch 9/200
1/1 - 0s - loss: 2.4042 - accuracy: 0.4545 - 3ms/epoch - 3ms/step
Epoch 10/200
1/1 - 0s - loss: 2.3924 - accuracy: 0.4545 - 2ms/epoch - 2ms/step
Epoch 11/200
1/1 - 0s - loss: 2.3801 - accuracy: 0.4545 - 3ms/epoch - 3ms/step
Epoch 12/200
1/1 - 0s - loss: 2.3672 - accuracy: 0.4545 - 3ms/epoch - 3ms/step
Epoch 13/200
1/1 - 0s - loss: 2.3536 - accuracy: 0.4545 -

Epoch 105/200
1/1 - 0s - loss: 0.7114 - accuracy: 0.7273 - 3ms/epoch - 3ms/step
Epoch 106/200
1/1 - 0s - loss: 0.7019 - accuracy: 0.7273 - 3ms/epoch - 3ms/step
Epoch 107/200
1/1 - 0s - loss: 0.6925 - accuracy: 0.7273 - 3ms/epoch - 3ms/step
Epoch 108/200
1/1 - 0s - loss: 0.6832 - accuracy: 0.7273 - 2ms/epoch - 2ms/step
Epoch 109/200
1/1 - 0s - loss: 0.6740 - accuracy: 0.7273 - 3ms/epoch - 3ms/step
Epoch 110/200
1/1 - 0s - loss: 0.6648 - accuracy: 0.7273 - 3ms/epoch - 3ms/step
Epoch 111/200
1/1 - 0s - loss: 0.6558 - accuracy: 0.7273 - 3ms/epoch - 3ms/step
Epoch 112/200
1/1 - 0s - loss: 0.6468 - accuracy: 0.7273 - 2ms/epoch - 2ms/step
Epoch 113/200
1/1 - 0s - loss: 0.6380 - accuracy: 0.7273 - 2ms/epoch - 2ms/step
Epoch 114/200
1/1 - 0s - loss: 0.6292 - accuracy: 0.7273 - 2ms/epoch - 2ms/step
Epoch 115/200
1/1 - 0s - loss: 0.6205 - accuracy: 0.7273 - 2ms/epoch - 2ms/step
Epoch 116/200
1/1 - 0s - loss: 0.6118 - accuracy: 0.7273 - 3ms/epoch - 3ms/step
Epoch 117/200
1/1 - 0s - loss: 0.6032 - 

In [26]:
# 생성하는 문장을 출력하는 함수를 통해 확인
def sentence_generation(model, tokenizer, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word
    sentence = ''

    # n번 반복
    for _ in range(n):
        # 현재 단어에 대한 정수 인코딩과 패딩
        encoded = tokenizer.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=5, padding='pre')
        # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장.
        result = model.predict(encoded, verbose=0)
        result = np.argmax(result, axis=1)

        for word, index in tokenizer.word_index.items(): 
            # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면 break
            if index == result:
                break

        # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        current_word = current_word + ' '  + word

        # 예측 단어를 문장에 저장
        sentence = sentence + ' ' + word

    sentence = init_word + sentence
    return sentence


In [31]:
print(sentence_generation(model, tokenizer, '경마장에', 4))

경마장에 있는 말이 뛰고 있다


In [32]:
print(sentence_generation(model, tokenizer, '그의', 2))

그의 말이 법이다


In [33]:
print(sentence_generation(model, tokenizer, '가는', 5))

가는 말이 고와야 오는 말이 곱다


### by LSTM

In [34]:
import pandas as pd
import numpy as np
from string import punctuation

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [35]:
df = pd.read_csv('ArticlesApril2018.csv')
df.head()

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleaders’ Settlement Offer: ...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"“I understand that they could meet with us, pa...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,What’s it like to eat at the second incarnatio...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...
3,5adf40d2068401528a2aa619,626,By JULIE HIRSCHFELD DAVIS and PETER BAKER,article,Unknown,"['Macron, Emmanuel (1977- )', 'Trump, Donald J...",68,Washington,0,2018-04-24 14:35:57,Europe,President Trump welcomed President Emmanuel Ma...,The New York Times,News,https://www.nytimes.com/2018/04/24/world/europ...
4,5adf3d64068401528a2aa60f,815,By IAN AUSTEN and DAN BILEFSKY,article,Unknown,"['Toronto, Ontario, Attack (April, 2018)', 'Mu...",68,Foreign,0,2018-04-24 14:21:21,Canada,"Alek Minassian, 25, a resident of Toronto’s Ri...",The New York Times,News,https://www.nytimes.com/2018/04/24/world/canad...


In [36]:
print('열의 개수: ',len(df.columns))
print(df.columns)

열의 개수:  15
Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')


In [37]:
# 학습에 사용할 headline에 null값이 있는지 확인
print(df['headline'].isnull().values.any())

False


In [38]:
# 헤드라인 열의 값들을 리스트로 저장
headline = []
# 헤드라인의 값들을 리스트로 저장
headline.extend(list(df.headline.values)) 
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

In [39]:
# 헤드라인이 unknown인 경우 제거
print('총 샘플의 개수 : {}'.format(len(headline)))

headline = [word for word in headline if word != "Unknown"]
print('노이즈값 제거 후 샘플의 개수 : {}'.format(len(headline)))

총 샘플의 개수 : 1324
노이즈값 제거 후 샘플의 개수 : 1214


In [40]:
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for Self-Expression?']

In [41]:
# 구두점 제거, 소문자화
def repreprocessing(raw_sentence):
    preproceseed_sentence = raw_sentence.encode("utf8").decode("ascii",'ignore')
    # 구두점 제거와 동시에 소문자화
    return ''.join(word for word in preproceseed_sentence if word not in punctuation).lower()

preprocessed_headline = [repreprocessing(x) for x in headline]
preprocessed_headline[:5]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression']

In [42]:
# 단어집합 생성
tokenizer = Tokenizer()
tokenizer.fit_on_texts(preprocessed_headline)
vocab_size = len(tokenizer.word_index) + 1
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 3494


In [43]:
# 정수인코딩 적용 후, 한 문장을 일부분씩 끊어서 훈련 데이터 구성
sequences = list()

for sentence in preprocessed_headline:

    # 각 샘플에 대한 정수 인코딩
    encoded = tokenizer.texts_to_sequences([sentence])[0] 
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

sequences[:11]

[[99, 269],
 [99, 269, 371],
 [99, 269, 371, 1115],
 [99, 269, 371, 1115, 582],
 [99, 269, 371, 1115, 582, 52],
 [99, 269, 371, 1115, 582, 52, 7],
 [99, 269, 371, 1115, 582, 52, 7, 2],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116],
 [100, 3]]

In [44]:
# 어떤 정수가 어떤 단어를 의미하는지 보여주는 dict index_to_word 생성
index_to_word = {}
for key, value in tokenizer.word_index.items(): # 인덱스를 단어로 바꾸기 위해 index_to_word를 생성
    index_to_word[value] = key

print('빈도수 상위 582번 단어 : {}'.format(index_to_word[582]))

빈도수 상위 582번 단어 : offer


In [46]:
# 패딩 작업 수행
max_len = max(len(l) for l in sequences)
print('샘플의 최대 길이 : {}'.format(max_len))

sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences[:3])

샘플의 최대 길이 : 24
[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0   99  269]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0   99  269  371]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0   99  269  371 1115]]


In [47]:
# 맨 우측 단어를 레이블로 분리
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]

print(y)

[ 269  371 1115 ...  115    2 1025]


In [48]:
# 레이블에 대해 one-hot encoding 수행
y = to_categorical(y, num_classes=vocab_size)

#### LSTM 모델 설계

In [50]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM

In [53]:
embedding_dim = 10
hidden_units = 128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=70, verbose=2)

Epoch 1/70
244/244 - 4s - loss: 7.6385 - accuracy: 0.0291 - 4s/epoch - 18ms/step
Epoch 2/70
244/244 - 3s - loss: 7.1154 - accuracy: 0.0322 - 3s/epoch - 13ms/step
Epoch 3/70
244/244 - 3s - loss: 6.9802 - accuracy: 0.0352 - 3s/epoch - 14ms/step
Epoch 4/70
244/244 - 4s - loss: 6.8545 - accuracy: 0.0417 - 4s/epoch - 16ms/step
Epoch 5/70
244/244 - 4s - loss: 6.6943 - accuracy: 0.0454 - 4s/epoch - 17ms/step
Epoch 6/70
244/244 - 5s - loss: 6.5007 - accuracy: 0.0496 - 5s/epoch - 22ms/step
Epoch 7/70
244/244 - 5s - loss: 6.2946 - accuracy: 0.0520 - 5s/epoch - 19ms/step
Epoch 8/70
244/244 - 6s - loss: 6.0816 - accuracy: 0.0609 - 6s/epoch - 25ms/step
Epoch 9/70
244/244 - 6s - loss: 5.8764 - accuracy: 0.0643 - 6s/epoch - 23ms/step
Epoch 10/70
244/244 - 5s - loss: 5.6839 - accuracy: 0.0710 - 5s/epoch - 20ms/step
Epoch 11/70
244/244 - 5s - loss: 5.5061 - accuracy: 0.0716 - 5s/epoch - 20ms/step
Epoch 12/70
244/244 - 5s - loss: 5.3376 - accuracy: 0.0813 - 5s/epoch - 21ms/step
Epoch 13/70
244/244 - 5s 

In [54]:
def sentence_generation(model, tokenizer, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word
    sentence = ''

    # n번 반복
    for _ in range(n):
        encoded = tokenizer.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=max_len-1, padding='pre')

        # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
        result = model.predict(encoded, verbose=0)
        result = np.argmax(result, axis=1)

        for word, index in tokenizer.word_index.items(): 
            # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
            if index == result:
                break

        # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        current_word = current_word + ' '  + word

        # 예측 단어를 문장에 저장
        sentence = sentence + ' ' + word

    sentence = init_word + sentence
    return sentence

In [55]:
print(sentence_generation(model, tokenizer, 'i', 10))

i colbert knows what north korea wants moscow fraud young back


In [56]:
print(sentence_generation(model, tokenizer, 'how', 10))

how to make a crossword puzzle economy a surplus of frowns
